# Traveling salesman problem


Das Ziel ist, den schnellsten Weg von der Startstadt über n Städte zur Zielstadt zu finden.

### City
Eine Klasse, welche die Eigentschaften einer Stadt definiert.

_*weitere Informationen sind im Codein den mit # markierten Zeilen zu finden*_

In [11]:
!pip -q install pyproj

In [12]:
class City:

    def __init__(self, name, lat, lon, temp):
        self.name = name
        self.lat = lat
        self.lon = lon
        self.temp = temp

    def describe(self):
        print("%s:  %.1f Lat,  %.1f Lon Temp %.1fC" %(self.name, self.lat, self.lon, self.temp))
    
    def info(self):
        print("%s:  %.1f Lat,  %.1f Lon" %(self.name, self.lat, self.lon))
   
    def __str__(self):

        #return "Stadt: %s    Koordinaten:(%.1f, %.1f)   Temperatur: %i" %(self.name, self.lat, self.lon, self.temp)
        return "%s" %(self.name)
   
    def __repr__(self):
        return self.__str__()
    

#### Beispiel

In [13]:
stg = City("Stg", 48.7758459, 9.1829321, 22)    #Stuttgart

In [14]:
print(stg)

Stg


Wenn man die return Ausgabe der Klasse City statt `return "%s" %(self.name)`
das Return
```
return "Stadt: %s    Koordinaten:(%.1f, %.1f)   Temperatur: %i" %(self.name, self.lat, self.lon, self.temp
```
verwendet, erhält man anstatt des oben stehenden Outputs folgenden Output:
```
Stadt: Stg           Koordinaten:(48.8, 9.2)              Temperatur: 2
```
(Diese Veränderung sorgt nur für einen informativeren Output,wird aber im eigentlichen Code nicht verwendet um andere Outputs simpler und lesbarer zu halten.)


### Berechnungen:
Der Abstand von 2 Städten wird berechnet.

In [15]:
import pyproj


def distance(city1,city2):
    geod = pyproj.Geod(ellps='WGS84')
    angle1,angle2,distance = geod.inv(city1.lon, city1.lat, city2.lon, city2.lat)
    return distance/1000.0

def tempDiff(city1, city2):
    return abs(city1.temp - city2.temp)


## Source:
(wird nicht immer ganz verwendet)


In [16]:
stg = City("Stg", 48.7758459, 9.1829321, 22)    #Stuttgart
ber = City("Ber", 52.521918,  13.413215, 21)    #Berlin
ham = City("Ham", 53.551085,  9.993682,  24)    #Hamburg
nür = City("Nür", 49.452030,  11.076750, 22)    #Nürnberg
fra = City("Fra", 50.110922,  8.682127,  23)    #Frankfurt
düs = City("Düs", 51.2277411, 6.7734556,  9)    #Düsseldorf
hbn = City("Hbn", 49.1426929, 9.210879,  10)    #Heilbronn
hdh = City("Hdh", 48.6893963, 10.1610948, 2)    #Heidenheim
drs = City("Drs", 51.0504088, 13.7372621, 4)    #Dresden
prs = City("Prs", 48.856614,  2.3522219,  3)    #Paris

# 1. Lösungsweg
Der 1. Lösungsweg ist im Vergleich zum 2. Lösungsweg (weiter unten zu finden) simpler aber dafür oft nicht der kürzeste Weg. 

### Meneken
Eine Klasse, welche die Eigentschafften von Meneken ( also der  sich bewegende) definiert.

In [17]:
class Meneken:
    def __init__(self, toDo, start, ziel, name):
        self.done = []
        self.toDo = toDo
        self.distanceDone = 0
        self.start = start 
        self.ziel = ziel 
        self.standort = start
        self.name = name
        self.sleeps = 0
    def info(self):
        print("Hi wir sind %s, sind gerade in %s, reisen von %s nach %s.Wir sind schon %.1f viel km gereist, haben %d mal übernachtet" 
            %(self.name, self.standort.name, self.start.name, self.ziel.name, self.distanceDone, self.sleeps))

    def move(self):
        if len(self.toDo) > 0:
            nextIndex = closestCity(self.standort, self.toDo)
            nextZiel = self.toDo.pop(nextIndex)
        else: 
            nextZiel = self.ziel
        distanze = distance(nextZiel,self.standort)
        self.distanceDone += distanze
        self.done.append(nextZiel)
        self.standort = nextZiel
        self.sleeps = len(self.done)
        
    def istAmZiel(self):
        return self.ziel in self.done

Der eigentliche Code

In [18]:

def closestCity(standort, ziele):
    shortest = ziele[0]
    shortestIdx = 0
    for t in range(1, len(ziele)):
        ziel1 = shortest
        ziel2 = ziele[t]
        if distance(standort, ziel1) > distance(standort, ziel2):
            shortest = ziel2
            shortestIdx = t

    return shortestIdx


jakiPapa = Meneken([ham, nür, fra, düs], stg, ber, "Jakob und Gregor")
while not(jakiPapa.istAmZiel()):
    jakiPapa.move()
    jakiPapa.info()


Hi wir sind Jakob und Gregor, sind gerade in Fra, reisen von Stg nach Ber.Wir sind schon 152.9 viel km gereist, haben 1 mal übernachtet
Hi wir sind Jakob und Gregor, sind gerade in Düs, reisen von Stg nach Ber.Wir sind schon 336.3 viel km gereist, haben 2 mal übernachtet
Hi wir sind Jakob und Gregor, sind gerade in Ham, reisen von Stg nach Ber.Wir sind schon 675.2 viel km gereist, haben 3 mal übernachtet
Hi wir sind Jakob und Gregor, sind gerade in Nür, reisen von Stg nach Ber.Wir sind schon 1137.4 viel km gereist, haben 4 mal übernachtet
Hi wir sind Jakob und Gregor, sind gerade in Ber, reisen von Stg nach Ber.Wir sind schon 1516.2 viel km gereist, haben 5 mal übernachtet


# 2. Lösungsweg
Der 2. Lösungsweg ist im Vergleich zum 1. Lösungsweg (weiter oben zu finden) komplexer und wesentlich preziser.

### Path
Eine Klasse, welche die Eigentschaften eines Weges definiert.

In [19]:
class Path:
    def __init__(self, todo, startCity,endCity ):
        self.visited = [startCity]  
        self.todo = todo 
        self.endCity = endCity
    def currDist(self):
        dist = 0
        for i in range(1,len(self.visited)):
            CityPrev = self.visited[i-1]
            CityI = self.visited[i]
            dist1 = distance(CityPrev,CityI)
            dist += dist1 
        dist += distance(self.visited[-1],self.endCity)
        return dist

    def __lt__(self,other):
        return self.currDist() < other.currDist()
        
        #konvertierung um stings mit print auszugeben
    def __str__(self):
        return "<visited: %s todo: %s End:%s, dist %.0f >" %(self.visited, self.todo, self.endCity, self.currDist())
    def __repr__(self):
        return self.__str__()
    


Der eigentliche Code

In [20]:
from copy import deepcopy
from time import time

alleTodo = [ham, nür, fra, düs, hbn, hdh, drs, prs]


numPath = 2000

start = time()

#numPfad = 3*len(alleTodo) 
allePfade = []

#generiere Pfad der in Stg anfangen
p = Path(alleTodo, stg, ber)
allePfade.append(p)

neuePfade = []

#hohlt bei einem Pfad eine City aus todo raus
for q in range(0,len(alleTodo)):
    for p in allePfade:
        for i in range(0,len(p.todo)):
            newP = deepcopy(p)
            cityNeu = newP.todo.pop(i)
            newP.visited.append(cityNeu)
            #newP zu neuePfade
            neuePfade.append(newP)
    neuePfade.sort()
    
    allePfade = neuePfade[:numPath]
    neuePfade = []

done = time()

allePfade.sort()
smalestDist = allePfade[0].currDist()
shortestIndex = 0

Die Prints

In [21]:
print(len(allePfade))
print("%.2fkm" %(smalestDist))
print("%.2fs Laufzeit" %(done - start))
print(allePfade[shortestIndex])

2000
2195.20km
46.39s Laufzeit
<visited: [Stg, Hbn, Hdh, Nür, Fra, Prs, Düs, Ham, Drs] todo: [] End:Ber, dist 2195 >
